# Import packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [10]:
print(df.columns)



Index(['policy_nr_hashed', 'welcome_discount', 'last_data_year',
       'first_data_year', 'churn', 'control_group', 'first_premium',
       'last_premium', 'first_split', 'last_split', 'last_customer_age',
       'last_accident_free_years', 'last_car_value', 'last_age_car',
       'last_brand', 'last_type', 'last_weight', 'last_fuel_type',
       'last_postcode', 'last_product', 'last_allrisk basis',
       'last_allrisk compleet', 'last_allrisk royaal', 'last_wa-extra',
       'last_wettelijke aansprakelijkheid', 'nr_cars', 'fake_alarm',
       'policyholder_change', 'max_nr_coverages', 'last_nr_coverages',
       'last_trend_nr_coverages', 'accident_years', 'last_year_car_change',
       'last_change_premium_abs', 'last_change_premium_perc',
       'years_since_last_car_change', 'n_last_vs_peak', 'last_vs_first_split',
       'lpa', 'cum_change_premium_abs', 'cum_change_premium_perc'],
      dtype='object')


# Data preparation

In [26]:
# Load your dataset
df = pd.read_csv("../data/prepped_data.csv", low_memory=False, index_col=0).drop_duplicates()



# # Assuming 'no WD and no LPA' represents untreated customers
df = df[df["welcome_discount"] == 1]


# # Check if the DataFrame is not empty
# if not untreated_df.empty:
#     # Define your features and target variable
#     X = untreated_df.drop(['policy_nr_hashed', 'last_data_year', 'churn', 'control_group', 'premiums', 'last_brand', 'last_type', 'last_fuel_type', 'last_postcode', 'last_product', 'last_trend_nr_coverages', 'last_change_premium_abs', 'last_change_premium_perc', 'years_since_last_car_change'], axis=1)
#     y = untreated_df['churn']

#     # Split the dataset into training and test sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# else:
#     print("No untreated customers found. Check the filtering criteria.")


(30151, 41)
(19661, 41)


In [21]:
categorical_features = []
continuous_features = []
binary_features = []

# Define a threshold for the maximum number of unique values for a categorical column
max_unique_values_for_categorical = 10

# Iterate through each column to determine if it's categorical, continuous, or binary
for column in df.columns:
    unique_values = df[column].nunique()
    if unique_values == 2:
        # If exactly 2 unique values, treat column as binary
        binary_features.append(column)
    elif (df[column].dtype == 'object' or unique_values <= max_unique_values_for_categorical) and unique_values > 2:
        # If object type or up to the threshold of unique values (and more than 2), treat as categorical
        categorical_features.append(column)
    else:
        # Otherwise, treat as continuous
        continuous_features.append(column)

print(f'Binary Features: {binary_features}')
print(f'Categorical Features: {categorical_features}')
print(f'Continuous Features: {continuous_features}')

for cat in categorical_features:
     df[cat] = df[cat].astype("category")

Binary Features: ['churn', 'last_allrisk basis', 'last_allrisk compleet', 'last_allrisk royaal', 'last_wa-extra', 'fake_alarm', 'policyholder_change', 'n_last_vs_peak', 'lpa']
Categorical Features: ['policy_nr_hashed', 'last_data_year', 'first_data_year', 'control_group', 'last_brand', 'last_type', 'last_fuel_type', 'last_product', 'nr_cars', 'max_nr_coverages', 'last_nr_coverages', 'last_trend_nr_coverages', 'last_year_car_change', 'years_since_last_car_change']
Continuous Features: ['welcome_discount', 'first_premium', 'last_premium', 'first_split', 'last_split', 'last_customer_age', 'last_accident_free_years', 'last_car_value', 'last_age_car', 'last_weight', 'last_postcode', 'last_wettelijke aansprakelijkheid', 'accident_years', 'last_change_premium_abs', 'last_change_premium_perc', 'last_vs_first_split', 'cum_change_premium_abs', 'cum_change_premium_perc']


In [22]:
# Assuming 'binary_features' and 'continuous_features' are lists of your binary and continuous columns
selected_features = binary_features + continuous_features

# Select only the binary and continuous features, excluding the target variable 'churn'
X = df[[col for col in df.columns if col in selected_features and col != "churn"]]
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


# Setting up GridsearchCV

In [54]:

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    # Add more parameters here if needed
}

# test

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV or BayesSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# bayes_search = BayesSearchCV(rf, search_spaces, n_iter=32, cv=5, n_jobs=-1) # Uncomment for BayesSearchCV



# Setting up Hyperopt

In [24]:
# # Define the search space
# space = {
#     'n_estimators': hp.choice('n_estimators', range(100, 301)),
#     'max_depth': hp.choice('max_depth', [None, 5, 10, 15]),
#     'min_samples_split': hp.choice('min_samples_split', range(2, 11))
# }

# # Define the objective function
# def objective(params):
#     rf = RandomForestClassifier(**params, random_state=42)
#     best_score = cross_val_score(rf, X_train, y_train, scoring='accuracy', cv=5).mean()
#     return {'loss': -best_score, 'status': STATUS_OK}

# # Run the optimization with a different method for setting the random state
# trials = Trials()
# best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=32, trials=trials, rstate=np.random.seed(42))

# # Convert index values to actual values
# best_params['n_estimators'] = range(100, 301)[best_params['n_estimators']]
# best_params['max_depth'] = [None, 5, 10, 15][best_params['max_depth']]
# best_params['min_samples_split'] = range(2, 11)[best_params['min_samples_split']]

# print("Best parameters:", best_params)

# Define the search space
space = {
    'n_estimators': hp.choice('n_estimators', range(100, 301)),
    'max_depth': hp.choice('max_depth', [None, 5, 10, 15]),
    'min_samples_split': hp.choice('min_samples_split', range(2, 11)),
    'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 6)),
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),  # Updated this line
    'max_leaf_nodes': hp.choice('max_leaf_nodes', [None, 10, 20, 30, 40, 50]),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    # 'class_weight': 'balanced' is set directly in the model initialization
}


# Define the objective function
def objective(params):
    # Include class_weight='balanced' in the model
    rf = RandomForestClassifier(**params, class_weight='balanced', random_state=42)
    best_score = cross_val_score(rf, X_train, y_train, scoring='accuracy', cv=5).mean()
    return {'loss': -best_score, 'status': STATUS_OK}

# Run the optimization
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=32, trials=trials, rstate=np.random.seed(42))

# Convert index values to actual values for the hyperparameters
best_params['n_estimators'] = range(100, 301)[best_params['n_estimators']]
best_params['max_depth'] = [None, 5, 10, 15, 20][best_params['max_depth']]
best_params['min_samples_split'] = range(2, 11)[best_params['min_samples_split']]
best_params['min_samples_leaf'] = range(1, 6)[best_params['min_samples_leaf']]
best_params['max_features'] = ['sqrt', 'log2', None][best_params['max_features']]
best_params['max_leaf_nodes'] = [None, 10, 20, 30, 40, 50][best_params['max_leaf_nodes']]
best_params['criterion'] = ['gini', 'entropy'][best_params['criterion']]

print("Best parameters:", best_params)



  0%|          | 0/32 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 32/32 [07:18<00:00, 13.72s/trial, best loss: -0.6685793352283766]
Best parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 30, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 175}


# Training the model

In [55]:
# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 300}
Best score: 0.9563075467419552


In [10]:
# Assuming 'best_params' contains the best parameters found by Hyperopt
# Create the RandomForestClassifier with the best parameters
rf_best = RandomForestClassifier(**best_params, random_state=42)

# Fit the model to the training data
rf_best.fit(X_train, y_train)

# Evaluate the model on the training set (optional, for comparison)
train_score = rf_best.score(X_train, y_train)
print("Training set score:", train_score)

# Evaluate the model on the test set
test_score = rf_best.score(X_test, y_test)
print("Test set score:", test_score)


Training set score: 0.8622127457948353
Test set score: 0.870218881273491


# Evaluate the model

In [12]:
# Use the best estimator to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)  # Ensure to use the transformed version of X_test if applicable

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
# Calculate metrics with 'weighted' average for multiclass classification
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision (Weighted): {precision}")
print(f"Recall (Weighted): {recall}")
print(f"F1 Score (Weighted): {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

NameError: name 'grid_search' is not defined

In [25]:
# Use the best parameters from Hyperopt and add class_weight='balanced'
rf_best = RandomForestClassifier(
    criterion='gini',
    n_estimators=245,
    max_depth=None,
    max_features='sqrt',
    max_leaf_nodes=30,
    min_samples_leaf=5,
    min_samples_split=8,
    class_weight='balanced',  # Adjust class weights
    random_state=42
)

# Fit the model to the training data
rf_best.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_best.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision (Weighted): {precision}")
print(f"Recall (Weighted): {recall}")
print(f"F1 Score (Weighted): {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.6467197830140702
Precision (Weighted): 0.8982995867789405
Recall (Weighted): 0.6467197830140702
F1 Score (Weighted): 0.7300143465709406
Confusion Matrix:
[[3514 1940]
 [ 144  301]]
